In [1]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient

In [2]:
MONGO_HOST= 'mongodb://127.0.0.1:27017/twitterdb'  # assuming you have mongoDB installed locally
                                             # and a database called 'twitterdb'

WORDS = ['#airpod','#ipad','#macbook','#iphone','#iwatch']

CONSUMER_KEY = "MhaxeyqFvFtSYareo0PHDoMa8"
CONSUMER_SECRET = "5QjuEKhbO5OvgV1Flsbs6XHKcsYgmKx4paSznl2W0Nwcydo6Q3"
ACCESS_TOKEN = "851594289205432321-ybASrG4uQ0ozHMLo7AprXwu3SgKNtye"
ACCESS_TOKEN_SECRET = "gll5tNhyz59KbZ1C1oOcQadPdh8HBS6REiyKwV7F0vQbC"


class StreamListener(tweepy.StreamListener):    
    #This is a class provided by tweepy to access the Twitter Streaming API. 

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        #This is the meat of the script...it connects to your mongoDB and stores the tweet
        try:
            client = MongoClient(MONGO_HOST)
            
            # Use twitterdb database. If it doesn't exist, it will be created.
            db = client.twitterdb
    
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']

            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            print('location:', datajson['user']['location'])
            print(datajson)
            print()
            print()
            print()
            
            #insert the data into the mongoDB into a collection called twitter_search
            #if twitter_search doesn't exist, it will be created.
            db.twitter_search.insert_one(datajson)
        except Exception as e:
           print(e)

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
streamer.filter(languages=["en"], track=WORDS)

Tracking: ['#airpod', '#ipad', '#macbook', '#iphone', '#iwatch']
You are now connected to the streaming API.


KeyboardInterrupt: 

In [166]:
client = MongoClient(MONGO_HOST)
            
            # Use twitterdb database. If it doesn't exist, it will be created.
db = client.twitterdb

cursor = db.twitter_search.find().limit(5000).sort([("created_at",-1)])

# cursor = db.collection.find().limit(200).sort([("created_at",-1)])

In [167]:
import pandas as pd
new_df = pd.DataFrame(columns = ['location'])
HASHTAGS = ['airpod','ipad','macbook','iphone','iwatch']

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
for post in cursor:
    if str(post['user']['location']).split(', ')[-1] in states:
        print('loc', str(post['user']['location']).split(', ')[-1])
        for i, hashtag in enumerate(post['entities']['hashtags']):
#             print(post['entities']['hashtags'][i]['text'])
            if post['entities']['hashtags'][i]['text'] in HASHTAGS:
                new_df = new_df.append({'location' : str(post['user']['location']).split(', ')[-1], 
                                        'time' : post['created_at'],
                                        'tag' : post['entities']['hashtags'][i]['text']},
                                         ignore_index=True)
                print('tag:', post['entities']['hashtags'][i]['text'])
    



loc NC
loc MA
loc GA
loc CA
loc TX
loc MI
loc LA
loc GA
loc MI
loc MA
loc CT
loc NY
loc CO
loc TX
loc UT
loc CA
loc NC
loc CA
loc IA
loc CA
loc CA
loc TX
loc TX
loc DC


In [168]:
new_df.describe()

,location
count,0
unique,0
